In [1]:
%load_ext autoreload
%autoreload 2

import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

from cv import CustomTimeSeriesSplit

## Generate some data

In [2]:
df = pd.read_csv('train.csv')
df['date'] = pd.to_datetime(df['date']).dt.date


In [3]:
df['container'].unique()

array(['glass', 'plastic', 'can', nan], dtype=object)

In [4]:
df.isna().sum()

id            0
date          0
city          0
lat          51
long         46
pop           0
shop          0
brand         0
container    16
capacity     15
price         0
quantity      0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isna().sum()

id           0
date         0
city         0
lat          0
long         0
pop          0
shop         0
brand        0
container    0
capacity     0
price        0
quantity     0
dtype: int64

In [7]:
df.set_index('date', inplace=True)

In [8]:
city_dict = df["city"].value_counts(normalize=True).to_dict()
brand_dict = df["brand"].value_counts(normalize=True).to_dict()
df['city'] = df['city'].map(city_dict)
df['brand'] = df['brand'].map(brand_dict)

In [9]:
shop_map = {'shop_1':0,'shop_2':2,'shop_6':1,
           'shop_4':3,'shop_3':4,'shop_5':5}
df['shop']= df['shop'].map(shop_map)

In [10]:
container_map={'glass':1,'plastic':2,'can':3}
df["container"]=df["container"].map(container_map)
df['capacity']=df['capacity'].str.replace('ml','')
df["capacity"]=df["capacity"].str.replace("lt","")

In [11]:
df.isna().sum()

id           0
city         0
lat          0
long         0
pop          0
shop         0
brand        0
container    0
capacity     0
price        0
quantity     0
dtype: int64

In [11]:
exclude_columns = ['id']
X = df.drop(columns=['quantity'] + exclude_columns)

In [13]:
X

city       lat      long     pop  shop     brand  container  \
date                                                                          
2012-01-31  0.332811  37.97945  23.71622  672130     0  0.200596          1   
2012-01-31  0.332811  37.97945  23.71622  672130     0  0.200596          2   
2012-01-31  0.332811  37.97945  23.71622  672130     0  0.200596          3   
2012-01-31  0.332811  37.97945  23.71622  672130     0  0.200282          1   
2012-01-31  0.332811  37.97945  23.71622  672130     0  0.200282          3   
...              ...       ...       ...     ...   ...       ...        ...   
2017-12-31  0.332811  37.96245  23.68708  665871     4  0.198871          2   
2017-12-31  0.166562  39.63689  22.41761  144302     5  0.198871          3   
2017-12-31  0.167033  38.24444  21.73444  168501     1  0.200282          1   
2017-12-31  0.167189  40.64361  22.93086  353001     3  0.199969          2   
2017-12-31  0.167189  40.64361  22.93086  353001     3  0.200282          1   

           capacity  price  
date                        
2012-01-31      500   0.96  
2012-01-31      1.5   2.86  
2012-01-31      330   0.87  
2012-01-31      500   1.00  
2012-01-31      330   0.39  
...             ...    ...  
2017-12-31      1.5   1.02  
2017-12-31      330   0.47  
2017-12-31      500   1.02  
2017-12-31      1.5   1.34  
2017-12-31      500   1.16  

[6376 rows x 9 columns]

In [12]:
y = df['quantity']

## Specify the sizes of train and test set and initialize the cross validators

In [13]:
outer_train_set_size = 280
outer_test_set_size = 70

inner_train_set_size = 140
inner_test_set_size = 70

outer_loop_splitter = CustomTimeSeriesSplit(
    train_set_size=outer_train_set_size,
    test_set_size=outer_test_set_size)

inner_loop_splitter = CustomTimeSeriesSplit(
    train_set_size=inner_train_set_size,
    test_set_size=inner_test_set_size)

## Initialize algorithms: pipeline + parameter grid

In [14]:
# Initialize models
model_1 = SVR()
model_2 = ElasticNet()

# Initialize pipelines
pipeline_1 = Pipeline([('scaling', StandardScaler()),
                       ('model', model_1)])
pipeline_2 = Pipeline([('scaling', StandardScaler()),
                       ('model', model_2)])

# Initialize hyperparameter grids
param_grid_1 = [{'model__kernel': ['rbf'],
                 'model__C': np.power(10., np.arange(-4, 4)),
                 'model__gamma': np.power(10., np.arange(-5, 0))},
                {'model__kernel': ['linear'],
                 'model__C': np.power(10., np.arange(-4, 4))}]
param_grid_2 = [{'model__l1_ratio': [.1, .5, .7, .9, .95, .99, 1],
                 'model__alpha': [.1, .5, 1, 10, 100]}]

# Make a DataFrame to save all algorithms and include columns to save the results
algorithms = {
        'param_grid': [param_grid_1, param_grid_2],                                            
        'pipeline': [pipeline_1, pipeline_2],
    }
algorithm_names = ['SVR', 'ElasticNet']
algorithms_and_results = pd.DataFrame(algorithms, index=algorithm_names)
for column in ['scores_cv_inner', 'scores_cv_outer']:
    algorithms_and_results[column] = [[] for _ in range(len(algorithm_names))]
algorithms_and_results

param_grid  \
SVR         [{'model__kernel': ['rbf'], 'model__C': [0.000...   
ElasticNet  [{'model__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95...   

                                    pipeline scores_cv_inner scores_cv_outer  
SVR                (StandardScaler(), SVR())              []              []  
ElasticNet  (StandardScaler(), ElasticNet())              []              []

## Run the nested cross validation and save the results

In [15]:
# The outer loop for algorithm selection and performance estimation
for outer_train_idx, outer_test_idx in outer_loop_splitter.split(X, y):
    x_train = X.iloc[outer_train_idx]
    y_train = y.iloc[outer_train_idx]
    x_test = X.iloc[outer_test_idx]
    y_test = y.iloc[outer_test_idx]
    print('-------Outer----------')
    print(x_train.index.min(),x_train.index.max())
    print(x_test.index.min(),x_test.index.max())
    # The grid search estimators need to be rebuild each outer fold, because the inner loop
    # cross validation can only be defined given the train set from the outer loop.
    grid_search_cv_estimators = {}
    for (name, param_grid, pipeline) in algorithms_and_results[['param_grid', 'pipeline']].itertuples():
        
        # The inner loop for hyperparameter tuning
        grid_search_cv_estimator = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring='r2',
            cv=inner_loop_splitter.split(x_train, y_train),
            n_jobs=4,
            refit=True)
        grid_search_cv_estimator.fit(x_train, y_train)

        outer_score = grid_search_cv_estimator.score(x_test, y_test)
        algorithms_and_results.loc[name, 'scores_cv_inner'].append(grid_search_cv_estimator.best_score_)
        algorithms_and_results.loc[name, 'scores_cv_outer'].append(outer_score)
algorithms_and_results

-------Outer----------
2012-04-30 2012-12-31
2013-01-31 2013-02-28
-------Outer----------
2012-06-30 2013-02-28
2013-03-31 2013-04-30
-------Outer----------
2012-08-31 2013-04-30
2013-05-31 2013-07-31
-------Outer----------
2012-10-31 2013-07-31
2013-08-31 2013-09-30
-------Outer----------
2013-01-31 2013-09-30
2013-10-31 2013-11-30
-------Outer----------
2013-03-31 2013-11-30
2013-12-31 2014-02-28
-------Outer----------
2013-05-31 2014-02-28
2014-03-31 2014-04-30
-------Outer----------
2013-08-31 2014-04-30
2014-05-31 2014-06-30
-------Outer----------
2013-10-31 2014-06-30
2014-07-31 2014-08-31
-------Outer----------
2013-12-31 2014-08-31
2014-09-30 2014-11-30
-------Outer----------
2014-03-31 2014-11-30
2014-12-31 2015-01-31
-------Outer----------
2014-05-31 2015-01-31
2015-02-28 2015-03-31
-------Outer----------
2014-07-31 2015-03-31
2015-04-30 2015-06-30
-------Outer----------
2014-09-30 2015-06-30
2015-07-31 2015-08-31
-------Outer----------
2014-12-31 2015-08-31
2015-09-30 2015-1

param_grid  \
SVR         [{'model__kernel': ['rbf'], 'model__C': [0.000...   
ElasticNet  [{'model__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95...   

                                    pipeline  \
SVR                (StandardScaler(), SVR())   
ElasticNet  (StandardScaler(), ElasticNet())   

                                              scores_cv_inner  \
SVR         [0.41434863258023613, 0.4527409532465745, 0.44...   
ElasticNet  [0.09613187405226886, 0.24019150745356588, 0.5...   

                                              scores_cv_outer  
SVR         [0.02969261781898458, 0.5366625822862046, 0.02...  
ElasticNet  [-0.6789453719414669, 0.5442968838430993, 0.28...

## Combine the results

In [16]:
for agg_function in [np.mean, np.median, np.std]:
    for column in ['scores_cv_outer', 'scores_cv_inner']:
        aggregation_dict = {column + '_' + agg_function.__name__: lambda df: df[column].map(agg_function)}
        algorithms_and_results = algorithms_and_results.assign(**aggregation_dict)
algorithms_and_results.T

SVR  \
param_grid              [{'model__kernel': ['rbf'], 'model__C': [0.000...   
pipeline                                        (StandardScaler(), SVR())   
scores_cv_inner         [0.41434863258023613, 0.4527409532465745, 0.44...   
scores_cv_outer         [0.02969261781898458, 0.5366625822862046, 0.02...   
scores_cv_outer_mean                                             0.289234   
scores_cv_inner_mean                                              0.23532   
scores_cv_outer_median                                           0.369884   
scores_cv_inner_median                                           0.346256   
scores_cv_outer_std                                              0.252148   
scores_cv_inner_std                                              0.245411   

                                                               ElasticNet  
param_grid              [{'model__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95...  
pipeline                                 (StandardScaler(), ElasticNet())  
scores_cv_inner         [0.09613187405226886, 0.24019150745356588, 0.5...  
scores_cv_outer         [-0.6789453719414669, 0.5442968838430993, 0.28...  
scores_cv_outer_mean                                             0.236778  
scores_cv_inner_mean                                             0.203894  
scores_cv_outer_median                                           0.313899  
scores_cv_inner_median                                           0.213659  
scores_cv_outer_std                                              0.374104  
scores_cv_inner_std                                              0.202265

## Select the best algorithm on metric of choice (here mean outer score)

In [17]:
best_algorithm_name = algorithms_and_results['scores_cv_outer_mean'].idxmax()
best_pipeline = algorithms_and_results.loc[best_algorithm_name, 'pipeline']
best_param_grid = algorithms_and_results.loc[best_algorithm_name, 'param_grid']
print(f'Choose best estimater: {best_algorithm_name}')

Choose best estimater: SVR


## Expected score

In [18]:
scores, mean_score = algorithms_and_results.loc[best_algorithm_name, ['scores_cv_outer', 'scores_cv_outer_mean']]
ax = sns.distplot(scores)
ax.set_title(f'Expected score for {best_algorithm_name}')
ax.axvline(mean_score, dashes=[6, 2], label=f'Mean score: {round(mean_score,2)}')
ax.legend();

/tmp/ipykernel_174219/2414368986.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  ax = sns.distplot(scores)
